# Error Modelling of Structural Growth Model for Lung Cancer in Absence of Treatment

The tumour growth inhibitory effects of Erlotinib and Gefitinib were modelled with a population PKPD model in [1]. A population PKPD model is a hierarchical model which consists of a structural model, a population model, and an error model. Each sub-model captures a different aspect of the tumour growth inhibition biology, and is parametrised by a set of parameters.

In this notebook we model the error of the structural lung cancer growth model in absence of treatment. We investigate the influence of the different noise model choices on the idenfifiability of the model, and employ visual, as well quantitative predictive checks to select the most promising error model. 

## Measured Tumour Growth in Absence of Treatment

In [1] patient-derived tumour explants LXF A677 (adenocarcinoma of the lung) were implanted in mice. The tumour growth was monitored over a period of 30 days, see the [Control Growth Data notebook](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/data_preparation.ipynb) for details.

In [2]:
#
# Visualise control growth data.
#

import os

import pandas as pd
import plotly.colors
import plotly.graph_objects as go


# Import data
# Get path of current working directory
path = os.getcwd()

# Import LXF A677 control growth data
data = pd.read_csv(path + '/data/lxf_control_growth.csv')

# Get mouse ids
mouse_ids = data['#ID'].unique()

# Get number of mice
n_mice = len(mouse_ids)

# Define colorscheme
colors = plotly.colors.qualitative.Plotly[:n_mice]

# Create figure
fig = go.Figure()

# Scatter plot LXF A677 time-series data for each mouse
for index, id_m in enumerate(mouse_ids):
    # Create mask for mouse
    mask = data['#ID'] == id_m

    # Get time points for mouse
    times = data['TIME in day'][mask]

    # Get observed tumour volumes for mouse
    observed_volumes = data['TUMOUR VOLUME in cm^3'][mask]

    # Get mass time series
    masses = data['BODY WEIGHT in g'][mask]

    # Plot tumour volume over time
    fig.add_trace(go.Scatter(
        x=times,
        y=observed_volumes,
        name="ID: %d" % id_m,
        showlegend=True,
        hovertemplate=
            "<b>ID: %d</b><br>" % (id_m) +
            "Time: %{x:} day<br>" +
            "Tumour volume: %{y:.02f} cm^3<br>" +
            "Body weight: %{text}<br>" +
            "Cancer type: Lung cancer (LXF A677)<br>" +
            "<extra></extra>",
        text=['%.01f g' % mass for mass in masses],
        mode="markers",
        marker=dict(
            symbol='circle',
            color=colors[index],
            opacity=0.7,
            line=dict(color='black', width=1))
    ))

    # Plot mass over time
    fig.add_trace(go.Scatter(
        x=times,
        y=masses,
        name="ID: %d" % id_m,
        showlegend=True,
        visible=False,
        hovertemplate=
            "<b>ID: %d</b><br>" % (id_m) +
            "Time: %{x:} day<br>" +
            "Tumour volume: %{y:.02f} cm^3<br>" +
            "Body weight: %{text}<br>" +
            "Cancer type: Lung cancer (LXF A677)<br>" +
            "<extra></extra>",
        text=['%.01f g' % mass for mass in masses],
        mode="markers",
        marker=dict(
            symbol='circle',
            color=colors[index],
            opacity=0.7,
            line=dict(color='black', width=1))
    ))

# Set X, Y axis and figure size
fig.update_layout(
    autosize=True,
    xaxis_title=r'$\text{Time in day}$',
    yaxis_title=r'$\text{Tumour volume in cm}^3$',
    template="plotly_white")

# Add switch between linear and log y-scale
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=[{"yaxis.type": "linear"}],
                    label="Linear y-scale",
                    method="relayout"
                ),
                dict(
                    args=[{"yaxis.type": "log"}],
                    label="Log y-scale",
                    method="relayout"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
        dict(
            type = "buttons",
            direction = "down",
            buttons=list([
                dict(
                    args=[
                        {"visible": [True, False] * n_mice},
                        {"yaxis": {"title": r'$\text{Tumour volume in cm}^3$'}}],
                    label="Tumour volume",
                    method="update"
                ),
                dict(
                    args=[
                        {"visible": [False, True] * n_mice}, 
                        {"yaxis": {"title": r'$\text{Body weight in g}$'}}],
                    label="Body weight",
                    method="update"
                ),
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=1.07,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()

**Figure 1 - Untreated tumour growth:** Untreated tumour growth of patient-derived tumour explants LXF A677 (adenocarcinoma of the lung) implanted in mice. The colouring of the data points indicates the identity of the mice. The evolution of the body weight can be explored by using the buttons in the top right.

## Structural model

In [[n1](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability_structural_model.ipynb)] we showed that a structural model with a transition from exponential growth of the tumour volume to a linear growth of the tumour volume is capable of capturing the observed data

\begin{equation*}
    \frac{\text{d}V^s_T}{\text{d}t} = \frac{\lambda V^s_T}{V^s_T / V_{\text{crit}} + 1}.
\end{equation*}

Here, $V^s_T$ is the tumour volume predicted by the structural model, $V_{\text{crit}}$ is the critical tumour volume at which the growth transitions from exponential to linear, and $\lambda $ is the exponential growth rate. We can see that the tumour growth is exponential for tumour volumes much smaller than the critical volume $V^s_T \ll V_{\text{crit}}$, and linear for tumour volumes much larger than the critical volume $V^s_T \gg V_{\text{crit}}$. This parameterisation is equivalent to the structural model presented in [1] when setting $\lambda $ equal to twice the exponential growth rate ($\lambda = 2\lambda _0$), and $\lambda V_{\text{crit}}$ equal to the linear growth rate ($\lambda V_{\text{crit}}=\lambda _0$).

In [[n1](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability_structural_model.ipynb)] we argued that the stability of the optimisation can be improved by non-dimensionalising the model, and log-transforming its parameteters. We thus perform all optimisations in this transformed search space.

## Constant noise model (all mice have their own noise parameter)

Explanation

- Identifiability, joined noise model

### Identifiability

In [1]:
#
# Define pints model wrapper such that myokit model can be used for inference.
#

import myokit
import pints

from pkpd import model as m


# Wrap myokit model, so it can be used with pints
class DimensionlessLogTransformedPintsModel(pints.ForwardModel):
    def __init__(self):
        # Create myokit model
        model = m.create_dimless_tumour_growth_model()

        # Create simulator
        self.sim = myokit.Simulation(model)

    def n_parameters(self):
        """
        Number of parameters to fit. Here log v, log a_0, log a_1.
        """
        return 3

    def n_outputs(self):
        return 1

    def simulate(self, log_parameters, times):
        # Reset simulation
        self.sim.reset()

        # Sort input parameters
        initial_volume, a_0, a_1 = np.exp(log_parameters)

        # Set initial condition
        self.sim.set_state([initial_volume])

        # Set growth constants
        self.sim.set_constant('central.a_0', a_0)
        self.sim.set_constant('central.a_1', a_1)

        # Define logged variable
        loggedVariable = 'central.volume_t'

        # Simulate
        output = self.sim.run(times[-1] + 1, log=[loggedVariable], log_times=times)
        result = output[loggedVariable]

        return np.array(result)

In [4]:
#
# Run optimisation multiple times from random initial starting points.
#
# This cell needs the above defined wrapped myokit model:
# [DimensionlessLogTransformedPintsModel]
#

import os

import myokit
import numpy as np
import pandas as pd
import pints
from tqdm.notebook import tqdm

import pkpd.likelihoods


# Define characteristic scales
characteristic_volume = 1  # in cm^3
characteristic_time = 1  # in day

# Define number of optimisation runs for each individual
n_runs = 10

# Import data
# Get path of current working directory
path = os.getcwd()

# Import LXF A677 control growth data
data = pd.read_csv(path + '/data/lxf_control_growth.csv')
n_mice = len(data['#ID'].unique())

# Define container for the structural model estimates
# Shape (n_mice, n_runs, n_structural_params + n_noise_params)
n_structural_params = 3
n_noise_params = 1
transf_params = np.empty(shape=(n_mice, n_runs, n_structural_params + n_noise_params))

# Define container for the objective function score for the optimised parameters
log_posterior_scores = np.empty(shape=(n_mice, n_runs))

# Create priors for tranformed parameters
log_prior_structural_params = pints.UniformLogPrior(
    lower_or_boundaries=np.log([1E-3] * n_structural_params),
    upper=np.log([1E3] * n_structural_params))
log_prior_sigma = pints.HalfCauchyLogPrior(location=0, scale=3)
log_prior = pints.ComposedLogPrior(log_prior_structural_params, log_prior_sigma)

# Create inverse problems for the LXF A677 population
mouse_ids = data['#ID'].unique()
for index, mouse_id in enumerate(tqdm(mouse_ids)):
    # Create mask for mouse with specfied ID
    mouse_mask = data['#ID'] == mouse_id

    # Get relevant time points
    times = data[mouse_mask]['TIME in day'].to_numpy() / characteristic_time

    # Get measured tumour volumes
    observed_volumes = data[mouse_mask]['TUMOUR VOLUME in cm^3'].to_numpy() / characteristic_volume

    # Create inverse problem
    problem = pints.SingleOutputProblem(
        DimensionlessLogTransformedPintsModel(), times, observed_volumes)

    # Create likelihood assuming a constant Gaussian noise
    log_likelihood = pints.GaussianLogLikelihood(problem)

    # Create posterior
    log_posterior = pints.LogPosterior(log_likelihood, log_prior)

    # Define initial starting points by sampling from prior
    initial_parameters = log_prior.sample(n=n_runs)

    # Run optimisation multiple times
    for run_id, initial_params in enumerate(initial_parameters):

        # Create optimisation controller with a CMA-ES optimiser
        optimiser = pints.OptimisationController(
            function=log_posterior,
            x0=initial_params,
            method=pints.CMAES)

        # Disable logging mode
        optimiser.set_log_to_screen(False)

        # Parallelise optimisation
        optimiser.set_parallel(True)

        # Find optimal parameters
        try:
            estimates, score = optimiser.run()
        except:
            # If inference breaks fill estimates with nan
            estimates = np.array([np.nan] * (n_structural_params + n_noise_params))
            score = np.nan

        # Save estimates and score
        transf_params[index, run_id, :] = estimates
        log_posterior_scores[index, run_id] = score

/Users/david/Documents/SABS/DPhil/Rotation 1 - PKPD Modelling Workflow/notebooks/ErlotinibGefitinib/venv/lib/python3.7/site-packages/cma/evolution_strategy.py:3597: RuntimeWarning:

invalid value encountered in double_scalars

/Users/david/Documents/SABS/DPhil/Rotation 1 - PKPD Modelling Workflow/notebooks/ErlotinibGefitinib/venv/lib/python3.7/site-packages/cma/evolution_strategy.py:3591: RuntimeWarning:

invalid value encountered in double_scalars




In [5]:
#
# Transform parameters to biological parameters (initial tumour volume, critical tumour volume, growth rate, sigma).
# 
# This cell needs the above inferred dimensionless parameters and the characteristic volume and time scale:
# [transf_params, characteristic_volume, characteristic_time]
#

import numpy as np


# Initialise container for backtransformed paramters
# Shape (n_mice, n_runs, n_parameters)
mouse_parameters = np.empty(shape=transf_params.shape)

# Transform initial volumes
# V_0 = exp(log(v_0)) * V^c
mouse_parameters[:, :, 0] = np.exp(transf_params[:, :, 0]) * characteristic_volume

# Transform critical volume 
# V_crit = exp(log(a_0)) * V^c
mouse_parameters[:, :, 1] = np.exp(transf_params[:, :, 1]) * characteristic_volume

# Transform exponential growth rate
# lambda = exp(log(a_1 / a_0) / t^c
mouse_parameters[:, :, 2] = \
    np.exp(transf_params[:, :, 2] - transf_params[:, :, 1]) / characteristic_time

# Transform sigma
# sigma = sigma_dimensionless * V^c
mouse_parameters[:, :, 3] = np.exp(transf_params[:, :, 3]) * characteristic_volume

In [6]:
#
# Visualisation of the spread of optimised model parameters for multiple runs from different initial points.
#
# This cell needs the above optimised parameters, and their respective objective function scores, as well as the data
# [mouse_parameters, log_posterior_scores, data]
#

import plotly.colors
import plotly.graph_objects as go


# Get mouse ids
mouse_ids = data['#ID'].unique()

# Get number of parameters + score (for visualisation)
n_params = mouse_parameters.shape[2] + 1

# Define colorscheme
colors = plotly.colors.qualitative.Plotly[:n_params]

# Get optimised parameters
optimised_parameters = mouse_parameters

# Get optimised parameters
scores = log_posterior_scores

# Create figure
fig = go.Figure()

# Box plot of optimised model parameters
for index, id_m in enumerate(mouse_ids):
    # Get optimised parameters
    parameters = optimised_parameters[index, ...]

    # Get scores
    score = scores[index, :]

    # Create box plot of for initial tumour volume
    fig.add_trace(
        go.Box(
            y=parameters[:, 0],  
            name="Initial tumour volume in cm^3",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[0],
            line_color=colors[0]))

    # Create box plot for critical volume
    fig.add_trace(
        go.Box(
            y=parameters[:, 1],  
            name="Critical volume in cm^3",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[1],
            line_color=colors[1]))

    # Create box plot of for exponential tumour growth
    fig.add_trace(
        go.Box(
            y=parameters[:, 2],  
            name="Exponential growth rate in 1/day",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[2],
            line_color=colors[2]))

    # Create box plot for sigma
    fig.add_trace(
        go.Box(
            y=parameters[:, 3],  
            name="Standard deviation in cm^3",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[3],
            line_color=colors[3]))
    
    # Create box plot of for objective function score
    fig.add_trace(
        go.Box(
            y=score,  
            name="Unnorm. log-posterior score",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[4],
            line_color=colors[4]))

# Set figure size
fig.update_layout(
    autosize=True,
    template="plotly_white")

# Add switch between mice
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "right",
            buttons=list([
                dict(
                    args=[{"visible": [True]*5 + [False]*(5 * 7)}],
                    label="ID: %d" % mouse_ids[0],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*5 + [True]*5 + [False]*(5 * 6)}],
                    label="ID: %d" % mouse_ids[1],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 2) + [True]*5 + [False]*(5 * 5)}],
                    label="ID: %d" % mouse_ids[2],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 3) + [True]*5 + [False]*(5 * 4)}],
                    label="ID: %d" % mouse_ids[3],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 4) + [True]*5 + [False]*(5 * 3)}],
                    label="ID: %d" % mouse_ids[4],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 5) + [True]*5 + [False]*(5 * 2)}],
                    label="ID: %d" % mouse_ids[5],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 6) + [True]*5 + [False]* 5}],
                    label="ID: %d" % mouse_ids[6],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 7) + [True]*5}],
                    label="ID: %d" % mouse_ids[7],
                    method="restyle"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()


** Figure 2 - Independent noise:** Scatter and box plot of the structural and error model parameter estimates $(\hat \psi , \hat \sigma )$ found by maximising the log-posterior for each mouse using a CMA-ES optimiser. The structural model parameters $\psi $ were assigned log-uniform priors, while the noise parameter $\sigma $ was assigned with a Half-Cauchy prior. For each mouse the optimisation routine was run 10 times from a starting point randomly sampled from the prior. In addition to the parameter estimates, also the distribution of the associated unnormalised log-posterior scores is presented.

Discussion

### Visual check: Validity of error model

- Predict tumour growth with structural model using median of optimised model parameters

- Plot
    1. Structural model predictions versus observations
    2. Residulas versus observations

- Both plots should include 1, 2, 3 sigma intervals of inferred model.

In [12]:
#
# Compute structural model predictions for each mouse.
#
# This cell needs the above optimised (non-dimensional) model parameters, the number of structural model parameters, and the characteristic time and volume scale:
# [transf_params, n_structural_params, characteristic_time, characteristic_volume]
#

import os

import numpy as np
import pandas as pd


# Import data
# Get path of current working directory
path = os.getcwd()

# Import LXF A677 control growth data
data = pd.read_csv(path + '/data/lxf_control_growth.csv')

# Get mouse IDs and times
mouse_ids_and_times = data[['#ID', 'TIME in day']]

# Get median parameters for each mouse
median_parameters = np.median(transf_params, axis=1)

# Instantiate model
model = DimensionlessLogTransformedPintsModel()

# Create container for simulated synthesised data
structural_model_predictions = []

# Simulate "noise-free" data
mouse_ids = mouse_ids_and_times['#ID'].unique()
for index, mouse_id in enumerate(mouse_ids):
    # Create mask for mouse
    mask = mouse_ids_and_times['#ID'] == mouse_id

    # Get times
    times = mouse_ids_and_times[mask]['TIME in day'].to_numpy() / characteristic_time

    # Get parameters
    parameters = median_parameters[index, :n_structural_params]

    # Predict volumes
    predicted_volumes = model.simulate(parameters, times) * characteristic_volume

    # Save dataframe
    df = pd.DataFrame({
        '#ID': mouse_ids_and_times[mask]['#ID'],
        'TIME in day': mouse_ids_and_times[mask]['TIME in day'],
        'PREDICTED TUMOUR VOLUME in cm^3': predicted_volumes})
    structural_model_predictions.append(df)

# Merge mouse dataframes to one dataframe
structural_model_predictions = pd.concat(structural_model_predictions)

In [45]:
#
# Visualise noise model.
#
# This cell needs the structural model predictions for the tumour growth, and the medians of the optimised parameters:
# [structural_model_predictions, median_parameters]
#

import os

import pandas as pd
import plotly.colors
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Import data
# Get path of current working directory
path = os.getcwd()

# Import LXF A677 control growth data
data = pd.read_csv(path + '/data/lxf_control_growth.csv')

# Match observations with predictions
data = data.merge(structural_model_predictions, on=['#ID', 'TIME in day'])

# Get noise parameters
sigmas = median_parameters[:, -1]

# Get mouse ids
mouse_ids = data['#ID'].unique()

# Get number of mice
n_mice = len(mouse_ids)

# Define colorscheme
colors = plotly.colors.qualitative.Plotly[:n_mice]

# Create figure
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.5, 0.5], vertical_spacing=0.05)

# Scatter plot LXF A677 time-series data for each mouse
for index, id_m in enumerate(mouse_ids):
    # Create mask for mouse
    mask = data['#ID'] == id_m

    # Get predicted tumour volumes for mouse
    predicted_volumes = data['PREDICTED TUMOUR VOLUME in cm^3'][mask]

    # Get observed tumour volumes for mouse
    observed_volumes = data['TUMOUR VOLUME in cm^3'][mask]

    # Get noise parameter
    sigma = sigmas[index]

    # Plot I: Measured vs predicted volumes
    # Plot measured tumour volume versus structural model tumour volume
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=observed_volumes,
            legendgroup="Measurement",
            name="Measurement",
            showlegend=True,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>ID: %d</b><br>" % (id_m) +
                "Structural model: %{x:.02f} cm^3<br>" +
                "Measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="markers",
            marker=dict(
                symbol='circle',
                color=colors[index],
                opacity=0.7,
                line=dict(color='black', width=1))),
        row=1,
        col=1)

    # Plot median of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=predicted_volumes,
            legendgroup="Error model",
            name="Error model",
            showlegend=True,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: Mean</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(color=colors[index]),
            opacity=0.7),
        row=1,
        col=1)

    # Plot 1-sigma interval of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=predicted_volumes + sigma,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 1-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.7),
        row=1,
        col=1)
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=predicted_volumes - sigma,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 1-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.7),
        row=1,
        col=1)

    # Plot 2-sigma interval of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=predicted_volumes + 2 * sigma,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 2-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black',
                width=1.5),
            opacity=0.5),
        row=1,
        col=1)
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=predicted_volumes - 2 * sigma,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 2-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black',
                width=1.5),
            opacity=0.5),
        row=1,
        col=1)

    # Plot 3-sigma interval of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=predicted_volumes + 3 * sigma,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 3-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black',
                width=1),
            opacity=0.3),
        row=1,
        col=1)
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=predicted_volumes - 3 * sigma,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 3-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black',
                width=1),
            opacity=0.3),
        row=1,
        col=1)

    # Plot II: Residuals vs predicted volumes
    # Plot residuals versus structural model tumour volume
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=observed_volumes - predicted_volumes,
            legendgroup="Measurement",
            name="Measurement",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>ID: %d</b><br>" % (id_m) +
                "Structural model: %{x:.02f} cm^3<br>" +
                "Residual: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="markers",
            marker=dict(
                symbol='circle',
                color=colors[index],
                opacity=0.7,
                line=dict(color='black', width=1))),
        row=2,
        col=1)

    # Plot median of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=np.full(shape=len(predicted_volumes), fill_value=0),
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: Mean</b><br>" +
                "Hypothetical residual: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(color=colors[index]),
            opacity=0.7),
        row=2,
        col=1)

    # Plot 1-sigma interval of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=np.full(shape=len(predicted_volumes), fill_value=sigma),
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 1-sigma interval</b><br>" +
                "Hypothetical residual: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.7),
        row=2,
        col=1)
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=np.full(shape=len(predicted_volumes), fill_value=-sigma),
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 1-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.7),
        row=2,
        col=1)

    # Plot 2-sigma interval of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=np.full(shape=len(predicted_volumes), fill_value=2 * sigma),
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 2-sigma interval</b><br>" +
                "Hypothetical residual: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.5),
        row=2,
        col=1)
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=np.full(shape=len(predicted_volumes), fill_value=-2 * sigma),
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 2-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.5),
        row=2,
        col=1)

    # Plot 3-sigma interval of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=np.full(shape=len(predicted_volumes), fill_value=3 * sigma),
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 3-sigma interval</b><br>" +
                "Hypothetical residual: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.3),
        row=2,
        col=1)
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=np.full(shape=len(predicted_volumes), fill_value=-3 * sigma),
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 3-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.3),
        row=2,
        col=1)

# Set figure size
fig.update_layout(
    autosize=True,
    template="plotly_white")

# Set X and Y axes
fig.update_xaxes(title_text=r'$\text{Structural model predictions in cm}^3$', row=2, col=1)
fig.update_yaxes(title_text=r'$\text{Tumour volume in cm}^3$', row=1, col=1)
fig.update_yaxes(title_text=r'$\text{Residuals in cm}^3$', row=2, col=1)


# Add switch between mice
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "right",
            buttons=list([
                dict(
                    args=[{"visible": [True]*(8 * 2) + [False]*(8 * 2 * 7)}],
                    label="ID: %d" % mouse_ids[0],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(8 * 2) + [True]*(8 * 2) + [False]*(8 * 2 * 6)}],
                    label="ID: %d" % mouse_ids[1],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(8 * 2 * 2) + [True]*(8 * 2) + [False]*(8 * 2 * 5)}],
                    label="ID: %d" % mouse_ids[2],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(8 * 2 * 3) + [True]*(8 * 2) + [False]*(8 * 2 * 4)}],
                    label="ID: %d" % mouse_ids[3],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(8 * 2 * 4) + [True]*(8 * 2) + [False]*(8 * 2 * 3)}],
                    label="ID: %d" % mouse_ids[4],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(8 * 2 * 5) + [True]*(8 * 2) + [False]*(8 * 2 * 2)}],
                    label="ID: %d" % mouse_ids[5],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(8 * 2 * 6) + [True]*(8 * 2) + [False]*(8 * 2)}],
                    label="ID: %d" % mouse_ids[6],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(8 * 2 * 7) + [True]*(8 * 2)}],
                    label="ID: %d" % mouse_ids[7],
                    method="restyle"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()

**Figure 3 - :**

Discussion:

- Correlation? 
- WAIC
- Symmetry test

## Multiplicative Gaussian error model (all mice have their own noise parameter)

Explanation

In [53]:
#
# Implements log-likelihood wrapper that allows fixing `eta` of `pints.MultiplicativeGaussianLogLikelihood`.
#

import pints

class FixedEtaMultiplicativeLogLikelihood(pints.LogPDF):
    """
    Implements log-likelihood wrapper for a `pints.MultiplicativeGaussianLogLikelihood`
    that allows fixing `eta`.
    """

    def __init__(self, log_likelihood, eta):
        if not isinstance(log_likelihood, pints.MultiplicativeGaussianLogLikelihood):
            raise ValueError(
                'This likelihood wrapper is only defined for a `pints.MultiplicativeLogLikelihood`.')
        if log_likelihood._problem.n_outputs() != 1:
            raise ValueError(
                'This likelihood wrapper is only defined for a `pints.SingleOutputProblem`.')
        
        self._log_pdf = log_likelihood
        self._eta = eta

    def __call__(self, parameters):
        # Create parameter container
        params = np.empty(shape=len(parameters)+1)

        # Fill container with parameters 
        # (Eta is at second last postion for SingleOutputProblems)
        params[:-2] = np.asarray(parameters[:-1])
        params[-2] = self._eta
        params[-1] = parameters[-1]

        return self._log_pdf(params)

    def n_parameters(self):
        return self._log_pdf.n_parameters() - 1


In [54]:
#
# Run optimisation multiple times from random initial starting points.
#
# This cell needs the above defined wrapped myokit model:
# [DimensionlessLogTransformedPintsModel]
#

import os

import myokit
import numpy as np
import pandas as pd
import pints
from tqdm.notebook import tqdm

import pkpd.likelihoods


# Define characteristic scales
characteristic_volume = 1  # in cm^3
characteristic_time = 1  # in day

# Define number of optimisation runs for each individual
n_runs = 10

# Import data
# Get path of current working directory
path = os.getcwd()

# Import LXF A677 control growth data
data = pd.read_csv(path + '/data/lxf_control_growth.csv')
n_mice = len(data['#ID'].unique())

# Define container for the structural model estimates
# Shape (n_mice, n_runs, n_structural_params + n_noise_params)
n_structural_params = 3
n_noise_params = 1
transf_params_multiplicative_noise = np.empty(
    shape=(n_mice, n_runs, n_structural_params + n_noise_params))

# Define container for the objective function score for the optimised parameters
log_posterior_scores_multiplicative_noise = np.empty(shape=(n_mice, n_runs))

# Create priors for tranformed parameters
log_prior_structural_params = pints.UniformLogPrior(
    lower_or_boundaries=np.log([1E-3] * n_structural_params),
    upper=np.log([1E3] * n_structural_params))
log_prior_sigma = pints.HalfCauchyLogPrior(location=0, scale=1)
log_prior = pints.ComposedLogPrior(log_prior_structural_params, log_prior_sigma)

# Create inverse problems for the LXF A677 population
mouse_ids = data['#ID'].unique()
for index, mouse_id in enumerate(tqdm(mouse_ids)):
    # Create mask for mouse with specfied ID
    mouse_mask = data['#ID'] == mouse_id

    # Get relevant time points
    times = data[mouse_mask]['TIME in day'].to_numpy() / characteristic_time

    # Get measured tumour volumes
    observed_volumes = data[mouse_mask]['TUMOUR VOLUME in cm^3'].to_numpy() / characteristic_volume

    # Create inverse problem
    problem = pints.SingleOutputProblem(
        DimensionlessLogTransformedPintsModel(), times, observed_volumes)

    # Create likelihood assuming a multiplicative Gaussian noise with fixed eta
    log_likelihood = pints.MultiplicativeGaussianLogLikelihood(problem)
    log_likelihood = FixedEtaMultiplicativeLogLikelihood(log_likelihood=log_likelihood, eta=1)

    # Create posterior
    log_posterior = pints.LogPosterior(log_likelihood, log_prior)

    # Define initial starting points by sampling from prior
    initial_parameters = log_prior.sample(n=n_runs)

    # Run optimisation multiple times
    for run_id, initial_params in enumerate(initial_parameters):

        # Create optimisation controller with a CMA-ES optimiser
        optimiser = pints.OptimisationController(
            function=log_posterior,
            x0=initial_params,
            method=pints.CMAES)

        # Disable logging mode
        optimiser.set_log_to_screen(False)

        # Parallelise optimisation
        optimiser.set_parallel(True)

        # Find optimal parameters
        try:
            estimates, score = optimiser.run()
        except:
            # If inference breaks fill estimates with nan
            estimates = np.array([np.nan] * (n_structural_params + n_noise_params))
            score = np.nan

        # Save estimates and score
        transf_params_multiplicative_noise[index, run_id, :] = estimates
        log_posterior_scores_multiplicative_noise[index, run_id] = score

/Users/david/Documents/SABS/DPhil/Rotation 1 - PKPD Modelling Workflow/notebooks/ErlotinibGefitinib/venv/lib/python3.7/site-packages/cma/evolution_strategy.py:3597: RuntimeWarning:

invalid value encountered in double_scalars




In [62]:
#
# Transform parameters to biological parameters (initial tumour volume, critical tumour volume, growth rate, sigma).
# 
# This cell needs the above inferred dimensionless parameters and the characteristic volume and time scale:
# [transf_params_multiplicative_noise, characteristic_volume, characteristic_time]
#

import numpy as np


# Initialise container for backtransformed paramters
# Shape (n_mice, n_runs, n_parameters)
mouse_parameters_multiplicative_noise = np.empty(shape=transf_params_multiplicative_noise.shape)

# Transform initial volumes
# V_0 = exp(log(v_0)) * V^c
mouse_parameters_multiplicative_noise[:, :, 0] = np.exp(transf_params_multiplicative_noise[:, :, 0]) * characteristic_volume

# Transform critical volume 
# V_crit = exp(log(a_0)) * V^c
mouse_parameters_multiplicative_noise[:, :, 1] = np.exp(transf_params_multiplicative_noise[:, :, 1]) * characteristic_volume

# Transform exponential growth rate
# lambda = exp(log(a_1 / a_0) / t^c
mouse_parameters_multiplicative_noise[:, :, 2] = \
    np.exp(transf_params_multiplicative_noise[:, :, 2] - transf_params_multiplicative_noise[:, :, 1]) / characteristic_time

# Transform sigma
# sigma = sigma_dimensionless * V^c
mouse_parameters_multiplicative_noise[:, :, 3] = \
    np.exp(transf_params_multiplicative_noise[:, :, 3]) * characteristic_volume

In [64]:
#
# Visualisation of the spread of optimised model parameters for multiple runs from different initial points.
#
# This cell needs the above optimised parameters, and their respective objective function scores, as well as the data
# [mouse_parameters_multiplicative_noise, log_posterior_scores_multiplicative_noise, data]
#

import plotly.colors
import plotly.graph_objects as go


# Get mouse ids
mouse_ids = data['#ID'].unique()

# Get number of parameters + score (for visualisation)
n_params = mouse_parameters.shape[2] + 1

# Define colorscheme
colors = plotly.colors.qualitative.Plotly[:n_params]

# Get optimised parameters
optimised_parameters = mouse_parameters_multiplicative_noise

# Get optimised parameters
scores = log_posterior_scores_multiplicative_noise

# Create figure
fig = go.Figure()

# Box plot of optimised model parameters
for index, id_m in enumerate(mouse_ids):
    # Get optimised parameters
    parameters = optimised_parameters[index, ...]

    # Get scores
    score = scores[index, :]

    # Create box plot of for initial tumour volume
    fig.add_trace(
        go.Box(
            y=parameters[:, 0],  
            name="Initial tumour volume in cm^3",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[0],
            line_color=colors[0]))

    # Create box plot for critical volume
    fig.add_trace(
        go.Box(
            y=parameters[:, 1],  
            name="Critical volume in cm^3",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[1],
            line_color=colors[1]))

    # Create box plot of for exponential tumour growth
    fig.add_trace(
        go.Box(
            y=parameters[:, 2],  
            name="Exponential growth rate in 1/day",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[2],
            line_color=colors[2]))

    # Create box plot for sigma
    fig.add_trace(
        go.Box(
            y=parameters[:, 3],  
            name="Rel. standard deviation in dimless",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[3],
            line_color=colors[3]))
    
    # Create box plot of for objective function score
    fig.add_trace(
        go.Box(
            y=score,  
            name="Unnorm. log-posterior score",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[4],
            line_color=colors[4]))

# Set figure size
fig.update_layout(
    autosize=True,
    template="plotly_white")

# Add switch between mice
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "right",
            buttons=list([
                dict(
                    args=[{"visible": [True]*5 + [False]*(5 * 7)}],
                    label="ID: %d" % mouse_ids[0],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*5 + [True]*5 + [False]*(5 * 6)}],
                    label="ID: %d" % mouse_ids[1],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 2) + [True]*5 + [False]*(5 * 5)}],
                    label="ID: %d" % mouse_ids[2],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 3) + [True]*5 + [False]*(5 * 4)}],
                    label="ID: %d" % mouse_ids[3],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 4) + [True]*5 + [False]*(5 * 3)}],
                    label="ID: %d" % mouse_ids[4],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 5) + [True]*5 + [False]*(5 * 2)}],
                    label="ID: %d" % mouse_ids[5],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 6) + [True]*5 + [False]* 5}],
                    label="ID: %d" % mouse_ids[6],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 7) + [True]*5}],
                    label="ID: %d" % mouse_ids[7],
                    method="restyle"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()


**Figure 4 - :**

### Visual check

In [67]:
#
# Compute structural model predictions for each mouse.
#
# This cell needs the above optimised (non-dimensional) model parameters, the number of structural model parameters, and the characteristic time and volume scale:
# [transf_params_multiplicative_noise, n_structural_params, characteristic_time, characteristic_volume]
#

import os

import numpy as np
import pandas as pd


# Import data
# Get path of current working directory
path = os.getcwd()

# Import LXF A677 control growth data
data = pd.read_csv(path + '/data/lxf_control_growth.csv')

# Get mouse IDs and times
mouse_ids_and_times = data[['#ID', 'TIME in day']]

# Get median parameters for each mouse
median_parameters_multiplicative_noise = np.median(
    transf_params_multiplicative_noise, axis=1)

# Instantiate model
model = DimensionlessLogTransformedPintsModel()

# Create container for simulated synthesised data
structural_model_predictions_multiplicative_noise = []

# Simulate "noise-free" data
mouse_ids = mouse_ids_and_times['#ID'].unique()
for index, mouse_id in enumerate(mouse_ids):
    # Create mask for mouse
    mask = mouse_ids_and_times['#ID'] == mouse_id

    # Get times
    times = mouse_ids_and_times[mask]['TIME in day'].to_numpy() / characteristic_time

    # Get parameters
    parameters = median_parameters_multiplicative_noise[index, :n_structural_params]

    # Predict volumes
    predicted_volumes = model.simulate(parameters, times) * characteristic_volume

    # Save dataframe
    df = pd.DataFrame({
        '#ID': mouse_ids_and_times[mask]['#ID'],
        'TIME in day': mouse_ids_and_times[mask]['TIME in day'],
        'PREDICTED TUMOUR VOLUME in cm^3': predicted_volumes})
    structural_model_predictions_multiplicative_noise.append(df)

# Merge mouse dataframes to one dataframe
structural_model_predictions_multiplicative_noise = pd.concat(
    structural_model_predictions_multiplicative_noise)

In [68]:
#
# Visualise noise model.
#
# This cell needs the structural model predictions for the tumour growth, and the medians of the optimised parameters:
# [structural_model_predictions_multiplicative_noise, median_parameters_multiplicative_noise]
#

import os

import pandas as pd
import plotly.colors
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Import data
# Get path of current working directory
path = os.getcwd()

# Import LXF A677 control growth data
data = pd.read_csv(path + '/data/lxf_control_growth.csv')

# Match observations with predictions
data = data.merge(structural_model_predictions_multiplicative_noise, on=['#ID', 'TIME in day'])

# Get noise parameters
sigmas = median_parameters_multiplicative_noise[:, -1]

# Get mouse ids
mouse_ids = data['#ID'].unique()

# Get number of mice
n_mice = len(mouse_ids)

# Define colorscheme
colors = plotly.colors.qualitative.Plotly[:n_mice]

# Create figure
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.5, 0.5], vertical_spacing=0.05)

# Scatter plot LXF A677 time-series data for each mouse
for index, id_m in enumerate(mouse_ids):
    # Create mask for mouse
    mask = data['#ID'] == id_m

    # Get predicted tumour volumes for mouse
    predicted_volumes = data['PREDICTED TUMOUR VOLUME in cm^3'][mask]

    # Get observed tumour volumes for mouse
    observed_volumes = data['TUMOUR VOLUME in cm^3'][mask]

    # Get noise parameter
    sigma = sigmas[index]

    # Plot I: Measured vs predicted volumes
    # Plot measured tumour volume versus structural model tumour volume
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=observed_volumes,
            legendgroup="Measurement",
            name="Measurement",
            showlegend=True,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>ID: %d</b><br>" % (id_m) +
                "Structural model: %{x:.02f} cm^3<br>" +
                "Measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="markers",
            marker=dict(
                symbol='circle',
                color=colors[index],
                opacity=0.7,
                line=dict(color='black', width=1))),
        row=1,
        col=1)

    # Plot median of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=predicted_volumes,
            legendgroup="Error model",
            name="Error model",
            showlegend=True,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: Mean</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(color=colors[index]),
            opacity=0.7),
        row=1,
        col=1)

    # Plot 1-sigma interval of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=predicted_volumes + sigma * predicted_volumes,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 1-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.7),
        row=1,
        col=1)
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=predicted_volumes - sigma * predicted_volumes,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 1-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.7),
        row=1,
        col=1)

    # Plot 2-sigma interval of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=predicted_volumes + 2 * sigma * predicted_volumes,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 2-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black',
                width=1.5),
            opacity=0.5),
        row=1,
        col=1)
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=predicted_volumes - 2 * sigma * predicted_volumes,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 2-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black',
                width=1.5),
            opacity=0.5),
        row=1,
        col=1)

    # Plot 3-sigma interval of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=predicted_volumes + 3 * sigma * predicted_volumes,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 3-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black',
                width=1),
            opacity=0.3),
        row=1,
        col=1)
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=predicted_volumes - 3 * sigma * predicted_volumes,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 3-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black',
                width=1),
            opacity=0.3),
        row=1,
        col=1)

    # Plot II: Residuals vs predicted volumes
    # Plot residuals versus structural model tumour volume
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=observed_volumes - predicted_volumes,
            legendgroup="Measurement",
            name="Measurement",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>ID: %d</b><br>" % (id_m) +
                "Structural model: %{x:.02f} cm^3<br>" +
                "Residual: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="markers",
            marker=dict(
                symbol='circle',
                color=colors[index],
                opacity=0.7,
                line=dict(color='black', width=1))),
        row=2,
        col=1)

    # Plot median of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=np.full(shape=len(predicted_volumes), fill_value=0),
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: Mean</b><br>" +
                "Hypothetical residual: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(color=colors[index]),
            opacity=0.7),
        row=2,
        col=1)

    # Plot 1-sigma interval of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=sigma * predicted_volumes,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 1-sigma interval</b><br>" +
                "Hypothetical residual: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.7),
        row=2,
        col=1)
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=-sigma * predicted_volumes,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 1-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.7),
        row=2,
        col=1)

    # Plot 2-sigma interval of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=2 * sigma * predicted_volumes,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 2-sigma interval</b><br>" +
                "Hypothetical residual: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.5),
        row=2,
        col=1)
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=-2 * sigma * predicted_volumes,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 2-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.5),
        row=2,
        col=1)

    # Plot 3-sigma interval of error model
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=3 * sigma * predicted_volumes,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 3-sigma interval</b><br>" +
                "Hypothetical residual: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.3),
        row=2,
        col=1)
    fig.add_trace(
        go.Scatter(
            x=predicted_volumes,
            y=-3 * sigma * predicted_volumes,
            legendgroup="Error model",
            name="Error model",
            showlegend=False,
            visible=True if index == 0 else False,
            hovertemplate=
                "<b>Error model: 3-sigma interval</b><br>" +
                "Hypothetical measurement: %{y:.02f} cm^3<br>" +
                "Cancer type: Lung cancer (LXF A677)<br>" +
                "<extra></extra>",
            mode="lines",
            line=dict(
                color='Black'),
            opacity=0.3),
        row=2,
        col=1)

# Set figure size
fig.update_layout(
    autosize=True,
    template="plotly_white")

# Set X and Y axes
fig.update_xaxes(title_text=r'$\text{Structural model predictions in cm}^3$', row=2, col=1)
fig.update_yaxes(title_text=r'$\text{Tumour volume in cm}^3$', row=1, col=1)
fig.update_yaxes(title_text=r'$\text{Residuals in cm}^3$', row=2, col=1)


# Add switch between mice
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "right",
            buttons=list([
                dict(
                    args=[{"visible": [True]*(8 * 2) + [False]*(8 * 2 * 7)}],
                    label="ID: %d" % mouse_ids[0],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(8 * 2) + [True]*(8 * 2) + [False]*(8 * 2 * 6)}],
                    label="ID: %d" % mouse_ids[1],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(8 * 2 * 2) + [True]*(8 * 2) + [False]*(8 * 2 * 5)}],
                    label="ID: %d" % mouse_ids[2],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(8 * 2 * 3) + [True]*(8 * 2) + [False]*(8 * 2 * 4)}],
                    label="ID: %d" % mouse_ids[3],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(8 * 2 * 4) + [True]*(8 * 2) + [False]*(8 * 2 * 3)}],
                    label="ID: %d" % mouse_ids[4],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(8 * 2 * 5) + [True]*(8 * 2) + [False]*(8 * 2 * 2)}],
                    label="ID: %d" % mouse_ids[5],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(8 * 2 * 6) + [True]*(8 * 2) + [False]*(8 * 2)}],
                    label="ID: %d" % mouse_ids[6],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(8 * 2 * 7) + [True]*(8 * 2)}],
                    label="ID: %d" % mouse_ids[7],
                    method="restyle"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()

**Figure 5 - :**

Discussion:
- Large "early" deviations push $\sigma _rel$ to larger values.
- Does not look like the correct model.
- Check whether WAIC favours constant Gaussian error over multiplicative error
- Log-posterior suggests:
    - ID 40: Constant Gaussian error
    - ID 94: Multiplicative Gaussian error
    - ID 95: Multiplicative Gaussian error (also changes inferred dynamics qualitatively from plain exponential growth to mixture ($V_{\text{crit}}$ from 1000 to 1))
    - ID 136: Multiplicative Gaussian error
    - ID 140: Multiplicative Gaussian error
    - ID 155: Multiplicative Gaussian error
    - ID 169: Multiplicative Gaussian error
    - ID 170: Constant Gaussian error

Does WAIC favour broader distributions? Worth checking.
    

## Combined Gaussian error model: Constant and multiplicative Gaussian error

In [ ]:
#
# Implements log-likelihood wrapper that allows fixing `eta` of `pints.MultiplicativeGaussianLogLikelihood`.
#

# TODO: not done!

import pints

class FixedEtaMultiplicativeLogLikelihood(ConstantAndMultiplicativeGaussianLoglikelihood):
    """
    Implements log-likelihood wrapper for a `pints.MultiplicativeGaussianLogLikelihood`
    that allows fixing `eta`.
    """

    def __init__(self, log_likelihood, eta):
        if not isinstance(log_likelihood, pints.MultiplicativeGaussianLogLikelihood):
            raise ValueError(
                'This likelihood wrapper is only defined for a `pints.MultiplicativeLogLikelihood`.')
        if log_likelihood._problem.n_outputs() != 1:
            raise ValueError(
                'This likelihood wrapper is only defined for a `pints.SingleOutputProblem`.')
        
        self._log_pdf = log_likelihood
        self._eta = eta

    def __call__(self, parameters):
        # Create parameter container
        params = np.empty(shape=len(parameters)+1)

        # Fill container with parameters 
        # (Eta is at second last postion for SingleOutputProblems)
        params[:-2] = np.asarray(parameters[:-1])
        params[-2] = self._eta
        params[-1] = parameters[-1]

        return self._log_pdf(params)

    def n_parameters(self):
        return self._log_pdf.n_parameters() - 1


## Constant Gaussian error model (shared noise model)

In [19]:
#
# Run optimisation multiple times from random initial starting points.
#
# This cell needs the above defined wrapped myokit model:
# [DimensionlessLogTransformedPintsModel]
#

import os

import myokit
import numpy as np
import pandas as pd
import pints
from tqdm.notebook import tqdm

import pkpd.likelihoods


# Define characteristic scales
characteristic_volume = 1  # in cm^3
characteristic_time = 1  # in day

# Define number of optimisation runs for each individual
n_runs = 10

# Import data
# Get path of current working directory
path = os.getcwd()

# Import LXF A677 control growth data
data = pd.read_csv(path + '/data/lxf_control_growth.csv')
n_mice = len(data['#ID'].unique())

# Define container for the structural model estimates
# Shape (n_mice, n_runs, n_structural_params + n_noise_params)
n_structural_params = 3
n_noise_params = 1
transf_params_shared_const_noise = np.empty(shape=(n_mice, n_runs, n_structural_params + n_noise_params))
total_n_parameters = n_mice * n_structural_params + n_noise_params

# Define container for the objective function score for the optimised parameters
scores_shared_const_noise = np.empty(shape=n_runs)

# Define random starting points over many orders of magnitude
# Shape = (n_runs, n_mice * n_parameters +1)
# For each mouse 3 structural parameters, and one shared noise parameter
initial_parameters = np.random.uniform(low=1E-3, high=1E1, size=(n_runs, total_n_parameters))

# Create inverse problems for the LXF A677 population
problems = []
mouse_ids = data['#ID'].unique()
for mouse_id in mouse_ids:
    # Create mask for mouse with specfied ID
    mouse_mask = data['#ID'] == mouse_id

    # Get relevant time points
    times = data[mouse_mask]['TIME in day'].to_numpy() / characteristic_time

    # Get measured tumour volumes
    observed_volumes = data[mouse_mask]['TUMOUR VOLUME in cm^3'].to_numpy() / characteristic_volume

    # Create inverse problem
    problems.append(
        pints.SingleOutputProblem(
            DimensionlessLogTransformedPintsModel(), times, observed_volumes))

# Create likelihood assuming a constant Gaussian noise
log_likelihood = pkpd.likelihoods.SharedNoiseLogLikelihood(
    problems=problems, log_likelihood=pints.GaussianLogLikelihood)

# Create prior to biologically relevant values
log_prior_structural_params = pints.UniformLogPrior(
    lower_or_boundaries=np.log([1E-3] * (total_n_parameters - 1)),
    upper=np.log([1E3] * (total_n_parameters - 1)))
log_prior_sigma = pints.HalfCauchyLogPrior(location=0, scale=3)
log_prior = pints.ComposedLogPrior(log_prior_structural_params, log_prior_sigma)

# Create posterior
log_posterior = pints.LogPosterior(log_likelihood, log_prior)

# Define initial starting points by sampling from prior
initial_parameters = log_prior.sample(n=n_runs)

# Run optimisation multiple times
for run_id, initial_params in enumerate(tqdm(initial_parameters)):

    # Create optimisation controller with a CMA-ES optimiser
    optimiser = pints.OptimisationController(
        function=log_posterior,
        x0=initial_params,
        method=pints.SNES)

    # Disable logging mode
    optimiser.set_log_to_screen(False)

    # Parallelise optimisation
    optimiser.set_parallel(True)

    # TODO: Remove this
    optimiser.set_max_iterations(1000)

    # Find optimal parameters
    try:
        estimates, score = optimiser.run()
    except:
        # If inference breaks fill estimates with nan
        estimates = np.array([np.nan] * total_n_parameters)
        score = np.nan

    # Save estimates and score
    for index, _ in enumerate(mouse_ids):
        # Match mice with structural model parameters
        transf_params_shared_const_noise[index, run_id, :-n_noise_params:] = estimates[
            index * n_structural_params:(index + 1) * n_structural_params]
    
    # Get noise parameter
    transf_params_shared_const_noise[:, run_id, -n_noise_params:] = np.array(np.repeat(
        a=estimates[-n_noise_params:, np.newaxis], repeats=n_mice, axis=0))
    scores_shared_const_noise[run_id] = score

In [20]:
#
# Transform parameters to biological parameters (initial tumour volume, critical tumour volume, growth rate, sigma).
# 
# This cell needs the above inferred dimensionless parameters and the characteristic volume and time scale:
# [transf_params_shared_const_noise, characteristic_volume, characteristic_time]
#

import numpy as np


# Initialise container for backtransformed paramters
# Shape (n_mice, n_runs, n_parameters)
mouse_params_shared_const_noise = np.empty(shape=transf_params_shared_const_noise.shape)

# Transform initial volumes
# V_0 = exp(log(v_0)) * V^c
mouse_params_shared_const_noise[:, :, 0] = np.exp(transf_params_shared_const_noise[:, :, 0]) * characteristic_volume

# Transform critical volume 
# V_crit = exp(log(a_0)) * V^c
mouse_params_shared_const_noise[:, :, 1] = np.exp(transf_params_shared_const_noise[:, :, 1]) * characteristic_volume

# Transform exponential growth rate
# lambda = exp(log(a_1 / a_0) / t^c
mouse_params_shared_const_noise[:, :, 2] = \
    np.exp(transf_params_shared_const_noise[:, :, 2] - transf_params_shared_const_noise[:, :, 1]) / characteristic_time

# Transform sigma
# sigma = sigma_dimensionless * V^c
mouse_params_shared_const_noise[:, :, 3] = \
    np.exp(transf_params_shared_const_noise[:, :, 3]) * characteristic_volume

In [23]:
#
# Visualisation of the spread of optimised model parameters for multiple runs from different initial points.
#
# This cell needs the above optimised parameters, and their respective objective function scores, as well as the data
# [mouse_params_shared_const_noise, scores_shared_const_noise, data]
#

import plotly.colors
import plotly.graph_objects as go


# Get mouse ids
mouse_ids = data['#ID'].unique()

# Get number of parameters + score (for visualisation)
n_params = mouse_params_shared_const_noise.shape[2] + 1

# Define colorscheme
colors = plotly.colors.qualitative.Plotly[:n_params]

# Get optimised parameters
optimised_parameters = mouse_params_shared_const_noise

# Get optimised parameters
scores = scores_shared_const_noise

# Create figure
fig = go.Figure()

# Box plot of optimised model parameters
for index, id_m in enumerate(mouse_ids):
    # Get optimised parameters
    parameters = optimised_parameters[index, ...]

    # Get scores
    score = scores

    # Create box plot of for initial tumour volume
    fig.add_trace(
        go.Box(
            y=parameters[:, 0],  
            name="Initial tumour volume in cm^3",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[0],
            line_color=colors[0]))

    # Create box plot for critical volume
    fig.add_trace(
        go.Box(
            y=parameters[:, 1],  
            name="Critical volume in cm^3",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[1],
            line_color=colors[1]))

    # Create box plot of for exponential tumour growth
    fig.add_trace(
        go.Box(
            y=parameters[:, 2],  
            name="Exponential growth rate in 1/day",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[2],
            line_color=colors[2]))

    # Create box plot for sigma
    fig.add_trace(
        go.Box(
            y=parameters[:, 3],  
            name="Standard deviation in cm^3",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[3],
            line_color=colors[3]))
    
    # Create box plot of for objective function score
    fig.add_trace(
        go.Box(
            y=score,  
            name="Score",
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True if index == 0 else False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[4],
            line_color=colors[4]))

# Set figure size
fig.update_layout(
    autosize=True,
    template="plotly_white")

# Add switch between mice
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "right",
            buttons=list([
                dict(
                    args=[{"visible": [True]*5 + [False]*(5 * 7)}],
                    label="ID: %d" % mouse_ids[0],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*5 + [True]*5 + [False]*(5 * 6)}],
                    label="ID: %d" % mouse_ids[1],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 2) + [True]*5 + [False]*(5 * 5)}],
                    label="ID: %d" % mouse_ids[2],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 3) + [True]*5 + [False]*(5 * 4)}],
                    label="ID: %d" % mouse_ids[3],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 4) + [True]*5 + [False]*(5 * 3)}],
                    label="ID: %d" % mouse_ids[4],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 5) + [True]*5 + [False]*(5 * 2)}],
                    label="ID: %d" % mouse_ids[5],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 6) + [True]*5 + [False]* 5}],
                    label="ID: %d" % mouse_ids[6],
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*(5 * 7) + [True]*5}],
                    label="ID: %d" % mouse_ids[7],
                    method="restyle"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()


In [5]:
log_likelihood._cum_n_problem_params

array([ 3,  6,  9, 12, 15, 18, 21, 24])

## Heteroscedastic noise model

Explanation

- Identifiability

## Linear heteroscedastic noise model

Explanation

- Identifiability

## Bibliography

- <a name="ref1"> [1] </a> Eigenmann et. al., Combining Nonclinical Experiments with Translational PKPD Modeling to Differentiate Erlotinib and Gefitinib, Mol Cancer Ther (2016)

## Notebook references

- <a name="ref1"> [n1] </a> [Identifiability of the Structural Model for Lung Cancer Growth in Absence of Treatment](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability_structural_model.ipynb)

[Back to project overview](https://github.com/DavAug/ErlotinibGefitinib/blob/master/README.md) | [Back to lung cancer control growth overview](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/overview.ipynb) | [Forward to next notebook](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/population_model.ipynb)